In [1]:
import os
import ast
import sys
import cv2
import json
import shutil
import random
import pickle
import imageio
import numpy as np
import pandas as pd
from PIL import Image
from collections import defaultdict
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
# sys.path.append('./')
# from utils import normalize_image, plot_image, plot_images, get_sequences

In [22]:
data_dir = '/data3/jupiter/datasets/nodar_datasets'
to_share_dir = os.path.join(data_dir, 'packed_data')
os.makedirs(to_share_dir, exist_ok=True)
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/11_3_rev1_train_human_test_dean_multires_ignore_trees_1p25_u_p5_h_p2_v_ft_from_rev1_22kdust_ft_p15dust_h_fresh_dust11/nodar_datasets'
datasets = os.listdir(data_dir)
datasets = [d for d in datasets if d != 'to_share']
print(len(datasets), datasets)

21 ['sequence_day_no_dust_driving', 'sequence_night_medium_dust_driving', 'sequence_day_low_dust_driving', 'sequence_dawn_dusk_low_dust_stationary', 'sequence_day_low_dust_stationary', 'sequence_night_heavy_dust_stationary', 'sequence_dawn_dusk_heavy_dust_stationary', 'sequence_dawn_dusk_medium_dust_stationary', 'sequence_night_heavy_dust_driving', 'sequence_night_low_dust_driving', 'sequence_dawn_dusk_no_dust_stationary', 'sequence_day_heavy_dust_stationary', 'sequence_day_heavy_dust_driving', 'sequence_night_low_dust_stationary', 'sequence_night_medium_dust_stationary', 'sequence_day_medium_dust_driving', 'sequence_dawn_dusk_heavy_dust_driving', 'sequence_day_medium_dust_stationary', 'sequence_day_no_dust_stationary', 'sequence_dawn_dusk_medium_dust_driving', 'sequence_dawn_dusk_low_dust_driving']


In [28]:
# read in master csv and select fields
converters = {"calibration_data": ast.literal_eval}
s = ''
for dataset in datasets:
    csv = os.path.join(data_dir, dataset, 'annotations.csv')
    df = pd.read_csv(csv, converters=converters)
    print(dataset, df.shape)
    # get intrinsics and select key columns
    df['camera_intrinsics'] = df['calibration_data'].apply(lambda d: d.get('intrinsics'))
    df2 = df[['id', 'frame_id', 'camera_location', 'camera_intrinsics']]
    # save csv
    dataset_dir = os.path.join(to_share_dir, dataset)
    os.makedirs(dataset_dir, exist_ok=True)
    df2.to_csv(os.path.join(dataset_dir, 'metadata.csv'), index=False)
    # save seg outputs
    if not os.path.isdir(os.path.join(dataset_dir, 'segmentation_prediction')):
        os.symlink(os.path.join(pred_dir, dataset, 'output_visualization'), os.path.join(dataset_dir, 'segmentation_prediction'), target_is_directory=True)
    # save raw tiff files
    if not os.path.isdir(os.path.join(dataset_dir, 'images')):
        os.symlink(os.path.join(data_dir, dataset, 'images'), os.path.join(dataset_dir, 'images'), target_is_directory=True)
    s += f'tar hcf {dataset}.tar {dataset}\n'
    # break
with open(os.path.join(to_share_dir, 'tar_files.sh'), 'w') as f:
    f.write(s)

sequence_day_no_dust_driving (1488, 116)
sequence_night_medium_dust_driving (584, 106)
sequence_day_low_dust_driving (559, 95)
sequence_dawn_dusk_low_dust_stationary (1488, 112)
sequence_day_low_dust_stationary (872, 112)
sequence_night_heavy_dust_stationary (1472, 115)
sequence_dawn_dusk_heavy_dust_stationary (1488, 106)
sequence_dawn_dusk_medium_dust_stationary (1470, 112)
sequence_night_heavy_dust_driving (1488, 106)
sequence_night_low_dust_driving (1498, 106)
sequence_dawn_dusk_no_dust_stationary (1472, 112)
sequence_day_heavy_dust_stationary (1176, 112)
sequence_day_heavy_dust_driving (1488, 115)
sequence_night_low_dust_stationary (1440, 115)
sequence_night_medium_dust_stationary (1488, 112)
sequence_day_medium_dust_driving (1200, 95)
sequence_dawn_dusk_heavy_dust_driving (812, 106)
sequence_day_medium_dust_stationary (1485, 112)
sequence_day_no_dust_stationary (1261, 95)
sequence_dawn_dusk_medium_dust_driving (1456, 106)
sequence_dawn_dusk_low_dust_driving (1116, 106)


In [11]:
'frame_id' in df

True

In [9]:
df.iloc[0].calibration_data

{'cal_height': 1204,
 'cal_width': 1944,
 'camera_idx': 7,
 'camera_location': 'T08',
 'extrinsics': {'r_rodrigues': '{"x": -0.3124042749404907, "y": 0.7538045048713684, "z": -0.5334541201591492, "w": -0.22273094952106476}',
  't': '{"x": -1.989351511001587, "y": 1.9224333763122559, "z": 1.1346606016159058}'},
 'gain': {'analog_gain': 1.0, 'digital_gain': 1.0},
 'intrinsics': {'d': [-0.29101812839508,
   0.13981582224369002,
   0.000166796438861,
   -5.3002084314357496e-05,
   -0.045897834002971004,
   0.0,
   0.0,
   0.0],
  'distortion_normalization': '{"normalization": 1459.33544921875, "center": {"x": 954.0071411132812, "y": 596.6936645507812}}',
  'k': [1460.3131103515625,
   0.0,
   954.0071411132812,
   0.0,
   1460.3131103515625,
   596.6936645507812,
   0.0,
   0.0,
   1.0]},
 'lighting_mode': None,
 'mode': None,
 'sensor_type': 'VD6763',
 'serial_number': 'PCE7T1B012313',
 'vpu_idx': 0}

In [17]:
df2.head(2)

,id,frame_id,camera_location,camera_intrinsics
0,672577b7123c6e5347e5b7dd,3647,T08,"{'d': [-0.29101812839508, 0.13981582224369002,..."
1,672577b8f13de5f8ea3caf16,3647,T14,"{'d': [-0.29000362753868103, 0.139604687690734..."


In [15]:
df2.groupby('frame_id').size()

frame_id
3623    16
3624    16
3625    16
3626    16
3627    16
        ..
3711    16
3712    16
3713    16
3714    16
3715    16
Length: 93, dtype: int64